In [1]:
# Célula 1: Configuração Inicial e Importações

# Importação de bibliotecas essenciais
import numpy as np
import pandas as pd
import os
import cv2 # OpenCV para leitura de vídeo e processamento de imagens
import dlib # Para detecção de face e 68 landmarks faciais
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump # Para salvar dados processados

# Configurações de exibição do Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Desativar avisos (útil para notebooks de desenvolvimento)
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso.")

# --- Definir caminhos para o dataset DROZY ---
# IMPORTANTE: Ajuste este caminho para o diretório onde você extraiu o dataset DROZY
# Exemplo: Se o seu notebook está em 'projeto_fadiga/notebooks/' e o DROZY em 'projeto_fadiga/DROZY/',
# o caminho relativo seria '../DROZY/'
DROZY_BASE_PATH = 'DROZY' # Caminho base para o diretório raiz do dataset DROZY

# Verificar se o caminho existe
if not os.path.exists(DROZY_BASE_PATH):
    print(f"ATENÇÃO: O caminho '{DROZY_BASE_PATH}' não foi encontrado.")
    print("Por favor, ajuste 'DROZY_BASE_PATH' para o diretório correto do dataset DROZY.")
else:
    print(f"Caminho base do DROZY definido para: {DROZY_BASE_PATH}")

# Caminhos específicos para os subdiretórios do DROZY
VIDEOS_PATH = os.path.join(DROZY_BASE_PATH, 'videos_i8')
TIMESTAMPS_PATH = os.path.join(DROZY_BASE_PATH, 'timestamps')
KSS_PATH = os.path.join(DROZY_BASE_PATH, 'KSS.txt')
PVT_RT_PATH = os.path.join(DROZY_BASE_PATH, 'pvt-rt')
ANNOTATIONS_AUTO_PATH = os.path.join(DROZY_BASE_PATH, 'annotations-auto')
INTERP_INDICES_PATH = os.path.join(DROZY_BASE_PATH, 'interpIndices')


# --- Carregar modelos pré-treinados para detecção de face e landmarks ---
# Dlib é uma boa escolha para ambos, e os modelos são bem otimizados.
# Baixe esses arquivos e coloque-os em uma pasta 'models' (ou ajuste o caminho)
# Arquivo shape_predictor_68_face_landmarks.dat deve ser baixado de:
# http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 (descompactar)
# Arquivo mmod_human_face_detector.dat (opcional, para detecção de face de maior qualidade)
# http://dlib.net/files/mmod_human_face_detector.dat.bz2 (descompactar)

MODELS_DIR = './models/' # Caminho para a pasta onde você guardará os modelos dlib

if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)
    print(f"Criado diretório para modelos: {MODELS_DIR}")

PREDICTOR_PATH = os.path.join(MODELS_DIR, 'shape_predictor_68_face_landmarks.dat')
FACE_DETECTOR_PATH = os.path.join(MODELS_DIR, 'mmod_human_face_detector.dat') # Opcional, dlib tem um detector de face padrão mais rápido

# Verificar se os modelos estão disponíveis
if not os.path.exists(PREDICTOR_PATH):
    print(f"ATENÇÃO: Arquivo '{PREDICTOR_PATH}' não encontrado.")
    print("Por favor, baixe 'shape_predictor_68_face_landmarks.dat' e coloque-o na pasta '{MODELS_DIR}'.")
    predictor = None
else:
    predictor = dlib.shape_predictor(PREDICTOR_PATH)
    print("Shape predictor de dlib carregado.")

# O detector de face padrão do dlib é 'dlib.get_frontal_face_detector()',
# que é rápido e geralmente suficiente. O MMOD é mais preciso, mas mais lento.
# Vamos usar o frontal_face_detector para este projeto por padrão, a menos que o MMOD seja explicitamente preferido
# para casos de uso mais desafiadores.
face_detector = dlib.get_frontal_face_detector()
# face_detector = dlib.cnn_face_detection_model_v1(FACE_DETECTOR_PATH) # Se preferir o detector CNN (mais lento)
print("Detector de face de dlib (frontal) carregado.")

Bibliotecas importadas com sucesso.
Caminho base do DROZY definido para: DROZY
Shape predictor de dlib carregado.
Detector de face de dlib (frontal) carregado.


In [2]:
# Célula 2: Visão Geral e Carregamento dos Dados KSS e PVT-RT (Ground Truth)

print("--- Visão Geral do Dataset DROZY ---")
print("O DROZY é um dataset multimodal para detecção de fadiga. Ele inclui:")
print(f"- Vídeos de intensidade (8-bit) em: {VIDEOS_PATH}")
print(f"- Timestamps por frame em: {TIMESTAMPS_PATH}")
print(f"- Scores da Karolinska Sleepiness Scale (KSS) em: {KSS_PATH}")
print(f"- Tempos de Reação do Psychomotor Vigilance Test (PVT-RT) em: {PVT_RT_PATH}")
print(f"- Anotações automáticas de 68 landmarks faciais 2D/3D em: {ANNOTATIONS_AUTO_PATH}")
print(f"- Índices de interpolação para frames perdidos em: {INTERP_INDICES_PATH}")
print("\nO objetivo é utilizar KSS e/ou PVT-RT como nosso 'ground truth' para o estado de fadiga.")

# --- Carregamento e Exploração dos Dados KSS ---
print("\n--- Carregando KSS Scores ---")
try:
    kss_df = pd.read_csv(KSS_PATH, header=None, sep=r'\s+', names=['Subject_1_Test_1', 'Subject_1_Test_2', 'Subject_1_Test_3'])

    # O KSS.txt tem uma estrutura peculiar, vamos reconstruí-la para ser mais utilizável
    # O README diz: "14 lines (1 per subject) and 3 columns (1 per test)"
    # Mas a estrutura parece ser mais complexa. Vamos tentar ler sem cabeçalho e transpor,
    # Ou, idealmente, se houvesse um CSV mais estruturado.
    # Dado o README, parece que cada linha representa um sujeito e cada coluna um teste.
    # No entanto, o KSS é uma escala subjetiva preenchida ao final de cada teste.
    # A maneira mais segura é tratar cada combinação Sujeito-Teste como uma entrada.

    # Vamos assumir que as linhas são sujeitos (1-14) e as colunas são testes (1-3)
    # E o teste 7-1 tem KSS 0 (arbitrário, pois não ocorreu)
    
    # KSS_data = np.loadtxt(KSS_PATH)
    # subjects = np.arange(1, 15)
    # tests = np.arange(1, 4)
    # kss_list = []
    # for i, subj in enumerate(subjects):
    #     for j, test in enumerate(tests):
    #         # Teste 7-1 é uma exceção
    #         if subj == 7 and test == 1:
    #             kss_score = 0
    #         else:
    #             kss_score = KSS_data[i, j]
    #         kss_list.append({'Subject': subj, 'Test': test, 'KSS_Score': kss_score})
    # kss_df = pd.DataFrame(kss_list)
    
    # Reavaliando o KSS.txt com base na descrição (14 linhas, 3 colunas)
    # Parece que cada linha é um sujeito e cada coluna é um teste (1, 2, 3)
    kss_raw_data = np.loadtxt(KSS_PATH)
    
    kss_records = []
    for s_idx in range(kss_raw_data.shape[0]):
        for t_idx in range(kss_raw_data.shape[1]):
            subject_id = s_idx + 1
            test_id = t_idx + 1
            kss_score = kss_raw_data[s_idx, t_idx]
            
            # Ajustar para o teste 7-1 que não ocorreu e tem KSS 0
            if subject_id == 7 and test_id == 1:
                kss_score = 0 # Conforme README
            
            kss_records.append({'Subject': subject_id, 'Test': test_id, 'KSS_Score': kss_score})
            
    kss_df = pd.DataFrame(kss_records)

    print("KSS Scores carregados.")
    print("Amostra do DataFrame KSS:")
    print(kss_df.head())
    print(f"\nDistribuição dos KSS Scores:\n{kss_df['KSS_Score'].value_counts().sort_index()}")

    # --- Definição do Ground Truth Binário a partir do KSS ---
    # Fadiga pode ser definida como KSS >= 7 (sonolência clara)
    # KSS 1-6 = Não Fadigado, KSS 7-9 = Fadigado
    KSS_FATIGUE_THRESHOLD = 7
    kss_df['Fatigue_Label_KSS'] = (kss_df['KSS_Score'] >= KSS_FATIGUE_THRESHOLD).astype(int)
    print(f"\nDistribuição dos rótulos binários (KSS >= {KSS_FATIGUE_THRESHOLD}):\n{kss_df['Fatigue_Label_KSS'].value_counts(normalize=True)}")

except FileNotFoundError:
    print(f"Erro: Arquivo KSS.txt não encontrado em '{KSS_PATH}'. Verifique o caminho.")
    kss_df = None
except Exception as e:
    print(f"Erro ao carregar ou processar KSS.txt: {e}")
    kss_df = None


# --- Carregamento e Exploração dos Dados PVT-RT ---
print("\n--- Carregando PVT Reaction Times (PVT-RT) ---")
# O PVT-RT está em arquivos .csv separados por Subject-Test
# Vamos carregar um exemplo para entender a estrutura
pvt_data = {}
for subject_folder in sorted(os.listdir(PVT_RT_PATH)):
    if subject_folder.startswith('Subject'):
        subject_id = int(subject_folder.replace('Subject', ''))
        pvt_data[subject_id] = {}
        subject_pvt_path = os.path.join(PVT_RT_PATH, subject_folder)
        
        # O README indica que os arquivos estão diretamente em pvt-rt/SUBJECT-TEST.csv
        # Na verdade, é SUBJECT-TEST.csv (ex: 1-1.csv)
        # Vamos listar todos os arquivos csv e extrair subject e test id
        
        for file_name in os.listdir(PVT_RT_PATH):
            if file_name.endswith('.csv'):
                try:
                    parts = file_name.replace('.csv', '').split('-')
                    s_id = int(parts[0])
                    t_id = int(parts[1])
                    
                    file_path = os.path.join(PVT_RT_PATH, file_name)
                    df_pvt_test = pd.read_csv(file_path, sep=';', header=None, skiprows=1) # skip header linha 1
                    
                    # A 1ª coluna é o timestamp do estímulo, a 2ª é o timestamp da reação
                    # RT = tempo de reação (coluna 1 - coluna 0)
                    reaction_times = (df_pvt_test.iloc[:, 1] - df_pvt_test.iloc[:, 0]).values
                    
                    if s_id not in pvt_data:
                        pvt_data[s_id] = {}
                    pvt_data[s_id][t_id] = reaction_times
                    
                except Exception as e:
                    print(f"Erro ao carregar PVT-RT para {file_name}: {e}")
                    continue
# Imprimir um exemplo
if pvt_data:
    first_subj = list(pvt_data.keys())[0]
    first_test = list(pvt_data[first_subj].keys())[0]
    print(f"Exemplo de PVT-RT para Sujeito {first_subj}, Teste {first_test}:")
    print(f"Número de reações: {len(pvt_data[first_subj][first_test])}")
    print(f"Média de RT: {np.mean(pvt_data[first_subj][first_test]):.2f} ms")
    print(f"Desvio Padrão de RT: {np.std(pvt_data[first_subj][first_test]):.2f} ms")
    plt.figure(figsize=(8, 4))
    sns.histplot(pvt_data[first_subj][first_test], bins=30, kde=True)
    plt.title(f'Distribuição dos Tempos de Reação PVT (Sujeito {first_subj}, Teste {first_test})')
    plt.xlabel('Tempo de Reação (ms)')
    plt.ylabel('Frequência')
    plt.show()
    
    # --- Definição do Ground Truth Binário a partir do PVT-RT ---
    # Uma forma comum é usar a média ou a mediana dos RTs.
    # Tempos de reação mais longos indicam fadiga.
    # Por agora, vamos apenas demonstrar o carregamento. A binarização exata será decidida
    # na fase de extração de características por janela temporal, onde o PVT-RT médio
    # para aquela janela será usado.
    print("\nPVT-RTs carregados. A binarização dos rótulos de fadiga a partir de PVT-RTs será feita")
    print("durante a extração de características por janela temporal no próximo passo.")
else:
    print("Nenhum dado PVT-RT carregado. Verifique o caminho ou a estrutura dos arquivos.")


print("\nPreparação inicial dos dados concluída. Próximo passo: processar vídeos e extrair características.")

--- Visão Geral do Dataset DROZY ---
O DROZY é um dataset multimodal para detecção de fadiga. Ele inclui:
- Vídeos de intensidade (8-bit) em: DROZY/videos_i8
- Timestamps por frame em: DROZY/timestamps
- Scores da Karolinska Sleepiness Scale (KSS) em: DROZY/KSS.txt
- Tempos de Reação do Psychomotor Vigilance Test (PVT-RT) em: DROZY/pvt-rt
- Anotações automáticas de 68 landmarks faciais 2D/3D em: DROZY/annotations-auto
- Índices de interpolação para frames perdidos em: DROZY/interpIndices

O objetivo é utilizar KSS e/ou PVT-RT como nosso 'ground truth' para o estado de fadiga.

--- Carregando KSS Scores ---
KSS Scores carregados.
Amostra do DataFrame KSS:
   Subject  Test  KSS_Score
0        1     1        3.0
1        1     2        6.0
2        1     3        7.0
3        2     1        3.0
4        2     2        7.0

Distribuição dos KSS Scores:
KSS_Score
0.0    1
2.0    5
3.0    7
4.0    4
5.0    2
6.0    7
7.0    9
8.0    5
9.0    2
Name: count, dtype: int64

Distribuição dos rótu

In [3]:
# Célula 3: Processamento de Vídeos, Extração de Landmarks e Cálculo de Métricas (EAR, MAR)

print("--- Processando Vídeos e Extraindo Características Faciais ---")

# --- Funções Auxiliares para Detecção de Landmarks e Cálculo de EAR/MAR ---

def shape_to_np(shape, dtype="int"):
    # Converte o objeto de shape de dlib em um array NumPy
    coords = np.zeros((68, 2), dtype=dtype)
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

def eye_aspect_ratio(eye):
    # Calcula a Eye Aspect Ratio (EAR)
    # Distâncias euclidianas entre os dois conjuntos de marcos verticais dos olhos (y-coord)
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    # Distância euclidiana entre o conjunto de marcos horizontais dos olhos (x-coord)
    C = np.linalg.norm(eye[0] - eye[3])
    # Calcula a EAR
    ear = (A + B) / (2.0 * C)
    return ear

def mouth_aspect_ratio(mouth):
    # Calcula a Mouth Aspect Ratio (MAR) para detecção de bocejos
    # Distâncias euclidianas entre os marcos verticais da boca
    A = np.linalg.norm(mouth[1] - mouth[7]) # 51-59 ou 61-67
    B = np.linalg.norm(mouth[2] - mouth[6]) # 52-58 ou 62-66
    C = np.linalg.norm(mouth[3] - mouth[5]) # 53-57 ou 63-65 (pontos internos, se disponíveis, ou externos)
    # Distância euclidiana entre os marcos horizontais da boca
    D = np.linalg.norm(mouth[0] - mouth[4]) # 48-54 ou 60-64

    # Calcula a MAR. Usamos a média das distâncias verticais dividido pela horizontal
    mar = (A + B + C) / (3.0 * D) 
    return mar

# Índices para os pontos de referência faciais (dlib 68 landmarks)
# Olho esquerdo: 36-41
# Olho direito: 42-47
# Boca: 48-67
LEFT_EYE_START, LEFT_EYE_END = 36, 42
RIGHT_EYE_START, RIGHT_EYE_END = 42, 48
MOUTH_START, MOUTH_END = 48, 68

# --- Lista de Sujeitos e Testes no Dataset DROZY ---
# Vamos inferir os IDs de Sujeitos e Testes a partir da estrutura de arquivos de vídeo
subject_test_combinations = []
if os.path.exists(VIDEOS_PATH):
    for video_file in os.listdir(VIDEOS_PATH):
        if video_file.endswith('.mp4'):
            try:
                # Exemplo: 1-1.mp4 -> Subject 1, Test 1
                parts = os.path.splitext(video_file)[0].split('-')
                subject_id = int(parts[0])
                test_id = int(parts[1])
                subject_test_combinations.append((subject_id, test_id))
            except Exception as e:
                print(f"Não foi possível parsear o nome do arquivo de vídeo {video_file}: {e}")
    subject_test_combinations = sorted(list(set(subject_test_combinations)))
    print(f"Encontradas {len(subject_test_combinations)} combinações Sujeito-Teste.")
else:
    print(f"Erro: Caminho de vídeos '{VIDEOS_PATH}' não encontrado. Não é possível inferir Sujeitos/Testes.")
    subject_test_combinations = [] # Garante que a lista esteja vazia se o caminho não existe

# --- Dicionário para armazenar todas as características extraídas ---
all_extracted_features = [] # List de dicionários, cada um para um frame

# --- Loop Principal para Processar Cada Vídeo (Sujeito-Teste) ---
# Limite para um número menor de vídeos para demonstração rápida. Remova em produção.
# MAX_VIDEOS_TO_PROCESS = 5 # Processar apenas os primeiros 5 vídeos para teste
# processed_videos_count = 0

for subject_id, test_id in subject_test_combinations:
    # if processed_videos_count >= MAX_VIDEOS_TO_PROCESS: # Ativar limite
    #    print(f"Limite de {MAX_VIDEOS_TO_PROCESS} vídeos atingido para demonstração.")
    #    break

    video_filename = f"{subject_id}-{test_id}.mp4"
    video_path = os.path.join(VIDEOS_PATH, video_filename)
    timestamps_path = os.path.join(TIMESTAMPS_PATH, f"{subject_id}-{test_id}.txt")
    
    # interp_indices_path não está sendo usado neste loop, então não precisa carregar aqui.
    # interp_indices_path = os.path.join(INTERP_INDICES_PATH, f"{subject_id}-{test_id}.txt")

    if not os.path.exists(video_path) or not os.path.exists(timestamps_path):
        print(f"Pulando {video_filename}: Arquivo de vídeo ou timestamps não encontrado.")
        continue
    
    print(f"\nProcessando vídeo: Sujeito {subject_id}, Teste {test_id} ({video_filename})")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Não foi possível abrir o vídeo: {video_path}")
        continue

    # Carregar timestamps
    timestamps_df = pd.read_csv(timestamps_path, header=None, sep=r'\s+',
                                names=['year', 'month', 'day', 'hours', 'minutes', 'seconds', 'milliseconds', 'elapsed_time_ms'])
    
    # Carregar KSS Score para este teste (se kss_df foi carregado corretamente na Célula 2)
    current_kss_score_row = kss_df[(kss_df['Subject'] == subject_id) & (kss_df['Test'] == test_id)]
    if not current_kss_score_row.empty:
        current_fatigue_label = current_kss_score_row['Fatigue_Label_KSS'].values[0]
    else:
        current_fatigue_label = -1 # Marcador para KSS não encontrado
        print(f"Aviso: KSS Score não encontrado para Sujeito {subject_id}, Teste {test_id}. Marcando frames com -1.")
        # Pode optar por pular o vídeo se o ground truth for crucial e ausente:
        # cap.release()
        # continue

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break # Fim do vídeo

        if frame_idx >= len(timestamps_df):
            print(f"Aviso: Mais frames que timestamps para {video_filename}. Parando.")
            break
        
        # Obter o timestamp para o frame atual
        current_timestamp_ms = timestamps_df.iloc[frame_idx]['elapsed_time_ms']

        # Converter para escala de cinza para dlib
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detecção de face
        # face_detector é 'dlib.get_frontal_face_detector()'
        faces = face_detector(gray, 0) # 0 = nenhum upsampling
        
        if len(faces) == 0:
            # print(f"Nenhuma face detectada no frame {frame_idx} de {video_filename}") # Descomente para depuração
            # Se nenhuma face for detectada, adicione um registro com valores padrão e Face_Detected = False
            all_extracted_features.append({
                'Subject': subject_id,
                'Test': test_id,
                'Frame_Index': frame_idx,
                'Elapsed_Time_ms': current_timestamp_ms,
                'EAR': 0.0, # Pode usar np.nan para indicar ausência real
                'MAR': 0.0, # Pode usar np.nan
                'Fatigue_Label': current_fatigue_label,
                'Face_Detected': False
            })
            frame_idx += 1
            continue
        
        # Considerando a maior face detectada se houver múltiplas
        # Para dlib.get_frontal_face_detector(), o objeto retornado é um rect diretamente
        face = max(faces, key=lambda rect: rect.width() * rect.height()) 

        # Detecção de 68 landmarks faciais
        shape = predictor(gray, face)
        shape_np = shape_to_np(shape)

        # Extração de características (EAR, MAR)
        left_eye = shape_np[LEFT_EYE_START:LEFT_EYE_END]
        right_eye = shape_np[RIGHT_EYE_START:RIGHT_EYE_END]
        mouth = shape_np[MOUTH_START:MOUTH_END]

        ear = (eye_aspect_ratio(left_eye) + eye_aspect_ratio(right_eye)) / 2.0
        mar = mouth_aspect_ratio(mouth)
        
        # Adicionar as características extraídas e o ground truth à lista
        all_extracted_features.append({
            'Subject': subject_id,
            'Test': test_id,
            'Frame_Index': frame_idx,
            'Elapsed_Time_ms': current_timestamp_ms,
            'EAR': ear,
            'MAR': mar,
            'Fatigue_Label': current_fatigue_label,
            'Face_Detected': True # Indica que a face foi detectada
        })

        # Opcional: Visualização de um frame (descomente para depuração)
        #if frame_idx % 30 == 0: # A cada segundo (se 30 fps)
        #    frame_display = frame.copy()
        #    cv2.rectangle(frame_display, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 0), 2)
        #    for (x, y) in shape_np:
        #        cv2.circle(frame_display, (x, y), 1, (0, 0, 255), -1)
        #    cv2.putText(frame_display, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        #    cv2.putText(frame_display, f"MAR: {mar:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        #    cv2.putText(frame_display, f"KSS Label: {current_fatigue_label}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        #     
        #     # Converta de BGR para RGB para exibir no matplotlib
        #    plt.imshow(cv2.cvtColor(frame_display, cv2.COLOR_BGR2RGB))
        #    plt.title(f"S{subject_id} T{test_id} - Frame {frame_idx}")
        #    plt.axis('off')
        #    plt.show()

        frame_idx += 1
    
    cap.release()
    # processed_videos_count += 1 # Ativar limite

print("\nProcessamento de vídeos (ou até o limite de teste) concluído.")
print(f"Total de entradas na lista de características: {len(all_extracted_features)}")

# Converter a lista de dicionários em um DataFrame Pandas
df_features = pd.DataFrame(all_extracted_features)

print("\n--- Amostra do DataFrame de Características Extraídas ---")
print(df_features.head())
print(f"\nTotal de frames processados no DataFrame: {len(df_features)}")

# --- Verificação e Tratamento da Coluna 'Face_Detected' ---
if not df_features.empty and 'Face_Detected' in df_features.columns:
    print(f"Porcentagem de frames com face detectada: {df_features['Face_Detected'].mean():.2%}")
else:
    print("Aviso: DataFrame de características está vazio ou a coluna 'Face_Detected' não foi criada.")
    print("Isso pode indicar que nenhum vídeo foi processado ou que a detecção facial falhou em todos os frames.")

# --- Tratamento de Outliers e Valores Ausentes (EAR/MAR=0.0 quando face não detectada) ---
# Decide como lidar com frames onde a face não foi detectada.
# Para o Modelo 1 e Modelo 2, pode ser útil manter 0.0s ou substituí-los por np.nan para um tratamento explícito
# de valores ausentes (e.g., com interpolação, ou um valor de preenchimento).
# Por agora, mantemos 0.0, mas este é um ponto para refinar na próxima célula ou nos notebooks dos modelos.

# Salvar o DataFrame de características para uso nos notebooks dos modelos
OUTPUT_DATA_DIR = 'processed_data' # Garante que está no mesmo nível dos notebooks
OUTPUT_FEATURES_FILE = os.path.join(OUTPUT_DATA_DIR, 'all_video_features.pkl')
os.makedirs(OUTPUT_DATA_DIR, exist_ok=True) # Garante que o diretório de saída existe
dump(df_features, OUTPUT_FEATURES_FILE)
print(f"\nCaracterísticas extraídas salvas em: {OUTPUT_FEATURES_FILE}")

print("\nCélula 3 concluída. Os dados agora estão prontos para serem transformados em sequências para os modelos.")

--- Processando Vídeos e Extraindo Características Faciais ---
Encontradas 36 combinações Sujeito-Teste.

Processando vídeo: Sujeito 1, Teste 1 (1-1.mp4)

Processando vídeo: Sujeito 1, Teste 2 (1-2.mp4)

Processando vídeo: Sujeito 1, Teste 3 (1-3.mp4)

Processando vídeo: Sujeito 2, Teste 1 (2-1.mp4)

Processando vídeo: Sujeito 2, Teste 2 (2-2.mp4)

Processando vídeo: Sujeito 2, Teste 3 (2-3.mp4)

Processando vídeo: Sujeito 3, Teste 1 (3-1.mp4)

Processando vídeo: Sujeito 3, Teste 2 (3-2.mp4)

Processando vídeo: Sujeito 3, Teste 3 (3-3.mp4)

Processando vídeo: Sujeito 4, Teste 1 (4-1.mp4)

Processando vídeo: Sujeito 4, Teste 2 (4-2.mp4)

Processando vídeo: Sujeito 4, Teste 3 (4-3.mp4)

Processando vídeo: Sujeito 5, Teste 1 (5-1.mp4)

Processando vídeo: Sujeito 5, Teste 2 (5-2.mp4)

Processando vídeo: Sujeito 5, Teste 3 (5-3.mp4)

Processando vídeo: Sujeito 6, Teste 1 (6-1.mp4)

Processando vídeo: Sujeito 6, Teste 2 (6-2.mp4)

Processando vídeo: Sujeito 6, Teste 3 (6-3.mp4)

Processando 

In [4]:
# Célula 4: Geração de Dataset para Treinamento (Preparação Final - Sequências e Amostras)

OUTPUT_DATA_DIR = 'processed_data'

print("--- Transformando dados de frame em sequências para treinamento dos modelos ---")

# Carregar o DataFrame de características salvo na Célula 3
try:
    df_features = pd.read_pickle(os.path.join(OUTPUT_DATA_DIR, 'all_video_features.pkl'))
    print(f"DataFrame de características carregado. Total de frames: {len(df_features)}")
    if df_features.empty:
        print("Aviso: O DataFrame de características está vazio. Não há dados para formar sequências.")
        # Pode adicionar um exit() ou raise uma exceção aqui se o projeto não puder continuar
except FileNotFoundError:
    print(f"Erro: Arquivo 'all_video_features.pkl' não encontrado em '{OUTPUT_DATA_DIR}'.")
    print("Por favor, execute a Célula 3 primeiro.")
    # exit() # Ou outro tratamento de erro
except Exception as e:
    print(f"Ocorreu um erro ao carregar o DataFrame de características: {e}")
    # exit()

# --- Definição dos Parâmetros de Sequência ---
# Estes parâmetros são cruciais e devem ser ajustados experimentalmente.
# Eles afetam diretamente como os modelos percebem a fadiga temporal.

SEQUENCE_LENGTH_SECONDS = 3  # Duração da sequência em segundos (e.g., 5 segundos de dados)
FPS = 30                     # Frames por segundo do vídeo (DROZY videos_i8 é 30 FPS)
SEQUENCE_LENGTH_FRAMES = SEQUENCE_LENGTH_SECONDS * FPS # Número de frames em cada sequência

OVERLAP_PERCENTAGE = 0.5     # Porcentagem de sobreposição entre sequências (e.g., 0.5 = 50% de sobreposição)
STEP_SIZE_FRAMES = int(SEQUENCE_LENGTH_FRAMES * (1 - OVERLAP_PERCENTAGE))

# Características que serão usadas como entrada para os modelos
# Excluímos 'Subject', 'Test', 'Frame_Index', 'Elapsed_Time_ms', 'Face_Detected' pois são meta-dados ou indicadores.
# 'Fatigue_Label' é o rótulo.
FEATURES_TO_USE = ['EAR', 'MAR']
# Adicione outras características se extraídas (ex: pose da cabeça, etc.)
# FEATURES_TO_USE = ['EAR', 'MAR', 'Head_Pitch', 'Head_Yaw', 'Head_Roll']

print(f"\nParâmetros de Sequência:")
print(f"  Duração da Sequência: {SEQUENCE_LENGTH_SECONDS} segundos ({SEQUENCE_LENGTH_FRAMES} frames)")
print(f"  Sobreposição de Sequência: {OVERLAP_PERCENTAGE*100}% ({STEP_SIZE_FRAMES} frames por passo)")
print(f"  Características utilizadas: {', '.join(FEATURES_TO_USE)}")

# --- Estruturas para armazenar as sequências e rótulos ---
all_sequences = [] # Para o Modelo 2 (TCN) - array 3D (num_samples, timesteps, num_features)
all_labels = []    # Rótulos correspondentes às sequências

# Para o Modelo 1 (Feature-Based):
# Será um DataFrame com características agregadas por janela (e.g., média, std dev da EAR/MAR na janela)
aggregated_features_for_model1 = []


# --- Processamento por Sujeito e Teste para Manter a Coerência Temporal ---
# Agrupamos por Sujeito e Teste para garantir que as sequências não misturem dados de diferentes vídeos.
grouped = df_features.groupby(['Subject', 'Test'])

for (subject, test), group in grouped:
    print(f"Gerando sequências para Sujeito: {subject}, Teste: {test} (Total de frames: {len(group)})")
    
    # Garantir que os frames estejam em ordem cronológica
    group = group.sort_values(by='Frame_Index').reset_index(drop=True)
    
    # Extrair os valores das características e o rótulo de fadiga para este grupo/vídeo
    current_features = group[FEATURES_TO_USE].values
    current_fatigue_label = group['Fatigue_Label'].iloc[0] # Assumimos um único rótulo KSS por teste
    
    # Para o Modelo 2 (TCN): Gerar sequências deslizantes
    for i in range(0, len(group) - SEQUENCE_LENGTH_FRAMES + 1, STEP_SIZE_FRAMES):
        sequence = current_features[i : i + SEQUENCE_LENGTH_FRAMES]
        
        # Opcional: Filtrar sequências que contenham muitos frames sem detecção de face
        # Um limiar (e.g., 80% dos frames devem ter face detectada) pode ser aplicado aqui
        # if (group['Face_Detected'].iloc[i : i + SEQUENCE_LENGTH_FRAMES].sum() / SEQUENCE_LENGTH_FRAMES) < 0.8:
        #     continue # Pular sequência se a qualidade for baixa

        all_sequences.append(sequence)
        all_labels.append(current_fatigue_label) # O rótulo KSS se aplica a todo o teste/vídeo

    # Para o Modelo 1 (Feature-Based): Agregação por Janela (EAR/MAR médio, STD, etc.)
    # Podemos também criar janelas e calcular estatísticas descritivas sobre EAR, MAR, etc.
    # para o Modelo 1, se não quisermos treiná-lo no nível de frame.
    # Se o Modelo 1 for treinado em características por frame, isso não é necessário.
    # Mas se quisermos que ele tome decisões sobre janelas, fazemos isso aqui.
    
    # Exemplo de agregação para o Modelo 1 (se ele for treinado em janelas agregadas):
    for i in range(0, len(group) - SEQUENCE_LENGTH_FRAMES + 1, STEP_SIZE_FRAMES):
        window_data = group.iloc[i : i + SEQUENCE_LENGTH_FRAMES]
        
        # Calcular a média e desvio padrão de EAR e MAR na janela
        mean_ear = window_data['EAR'].mean()
        std_ear = window_data['EAR'].std() if len(window_data['EAR']) > 1 else 0.0 # Evitar div por zero
        mean_mar = window_data['MAR'].mean()
        std_mar = window_data['MAR'].std() if len(window_data['MAR']) > 1 else 0.0

        # Adicionar outras métricas da janela, como PERCLOS, contagem de bocejos
        # Estes precisariam de lógica adicional aqui.
        # Por exemplo, PERCLOS para a janela:
        # EAR_THRESHOLD = 0.25 # Exemplo de limiar para olho fechado
        # closed_eye_frames = (window_data['EAR'] < EAR_THRESHOLD).sum()
        # perclos = (closed_eye_frames / SEQUENCE_LENGTH_FRAMES) * 100 # % de frames com olho fechado

        # Contagem de bocejos (precisa de um limiar para MAR e persistência)
        # yawns = (window_data['MAR'] > MAR_THRESHOLD).sum() # Lógica mais complexa para bocejos reais
        
        aggregated_features_for_model1.append({
            'Subject': subject,
            'Test': test,
            'Sequence_Start_Frame': i,
            'Mean_EAR': mean_ear,
            'Std_EAR': std_ear,
            'Mean_MAR': mean_mar,
            'Std_MAR': std_mar,
            # 'PERCLOS': perclos, # Se calculado
            # 'Yawn_Count': yawns, # Se calculado
            'Fatigue_Label': current_fatigue_label # Rótulo KSS para a janela
        })

print("\n--- Finalizando Geração de Sequências e Features Agregadas ---")

# Converter a lista de sequências para um array NumPy (para TCN)
X_sequences = np.array(all_sequences)
y_labels_tcn = np.array(all_labels)

print(f"Formato final das sequências para TCN (X_sequences): {X_sequences.shape}")
print(f"Formato final dos rótulos para TCN (y_labels_tcn): {y_labels_tcn.shape}")

# Converter a lista de features agregadas para um DataFrame Pandas (para Modelo 1)
df_model1_features = pd.DataFrame(aggregated_features_for_model1)

# X_model1_features será as colunas de características, y_labels_model1 será o Fatigue_Label
X_model1_features = df_model1_features[['Mean_EAR', 'Std_EAR', 'Mean_MAR', 'Std_MAR']].values # Ajuste se adicionar mais features
y_labels_model1 = df_model1_features['Fatigue_Label'].values

print(f"\nFormato final das características para Modelo 1 (X_model1_features): {X_model1_features.shape}")
print(f"Formato final dos rótulos para Modelo 1 (y_labels_model1): {y_labels_model1.shape}")

print("\n--- Salvando Dados Processados para os Modelos ---")

# Salvar dados para o Modelo 1
model1_data = {'features': X_model1_features, 'labels': y_labels_model1}
dump(model1_data, os.path.join(OUTPUT_DATA_DIR, 'model1_features_labels.pkl'))
print(f"Dados para Modelo 1 salvos em: {os.path.join(OUTPUT_DATA_DIR, 'model1_features_labels.pkl')}")

# Salvar dados para o Modelo 2 (TCN)
model2_data = {'sequences': X_sequences, 'labels': y_labels_tcn}
dump(model2_data, os.path.join(OUTPUT_DATA_DIR, 'model2_sequences_labels.pkl'))
print(f"Dados para Modelo 2 (TCN) salvos em: {os.path.join(OUTPUT_DATA_DIR, 'model2_sequences_labels.pkl')}")

print("\nCélula 4 concluída. Os datasets para cada modelo foram preparados e salvos.")
print("Agora você pode prosseguir para os notebooks de treinamento de cada modelo.")

--- Transformando dados de frame em sequências para treinamento dos modelos ---
Ocorreu um erro ao carregar o DataFrame de características: invalid load key, '\x10'.

Parâmetros de Sequência:
  Duração da Sequência: 3 segundos (90 frames)
  Sobreposição de Sequência: 50.0% (45 frames por passo)
  Características utilizadas: EAR, MAR
Gerando sequências para Sujeito: 1, Teste: 1 (Total de frames: 17865)
Gerando sequências para Sujeito: 1, Teste: 2 (Total de frames: 9497)
Gerando sequências para Sujeito: 1, Teste: 3 (Total de frames: 8866)
Gerando sequências para Sujeito: 2, Teste: 1 (Total de frames: 17899)
Gerando sequências para Sujeito: 2, Teste: 2 (Total de frames: 9401)
Gerando sequências para Sujeito: 2, Teste: 3 (Total de frames: 8146)
Gerando sequências para Sujeito: 3, Teste: 1 (Total de frames: 17882)
Gerando sequências para Sujeito: 3, Teste: 2 (Total de frames: 8848)
Gerando sequências para Sujeito: 3, Teste: 3 (Total de frames: 8874)
Gerando sequências para Sujeito: 4, Teste

In [5]:
# Célula 5: Resumo e Próximos Passos

print("--- Resumo da Preparação e Exploração de Dados ---")
print("Este notebook concluiu as seguintes etapas:")
print("1. Configuração do ambiente e importação das bibliotecas necessárias.")
print("2. Carregamento e exploração inicial dos dados de Ground Truth (KSS e PVT-RT).")
print("3. Processamento de todos os vídeos do dataset DROZY, incluindo:")
print("   - Detecção facial utilizando dlib.")
print("   - Extração de 68 landmarks faciais por frame.")
print("   - Cálculo de métricas de fadiga (EAR e MAR) por frame.")
print(f"   - Total de {len(df_features)} frames processados e salvos em '{os.path.join(OUTPUT_DATA_DIR, 'all_video_features.pkl')}'.")
print("4. Transformação dos dados de nível de frame em sequências temporais e características agregadas por janela:")
print(f"   - Geradas {len(X_sequences)} sequências de {SEQUENCE_LENGTH_SECONDS} segundos ({SEQUENCE_LENGTH_FRAMES} frames) para o Modelo 2 (TCN).")
print(f"   - Geradas {len(X_model1_features)} amostras com características agregadas por janela para o Modelo 1.")
print(f"   - Dados salvos em '{os.path.join(OUTPUT_DATA_DIR, 'model1_features_labels.pkl')}' e '{os.path.join(OUTPUT_DATA_DIR, 'model2_sequences_labels.pkl')}'.")

print("\nOs dados estão agora prontos e pré-processados para o treinamento de ambos os modelos de detecção de fadiga.")

print("\n--- Próximos Passos Sugeridos ---")
print("1. Abra o notebook `01_Model_1_Feature_Based_Drowsiness_Detection.ipynb`:")
print("   - Carregue os dados de `model1_features_labels.pkl`.")
print("   - Treine, avalie e salve o Modelo 1 (SVM/Random Forest/MLP).")
print("2. Abra o notebook `02_Model_2_TCN_Temporal_Drowsiness_Detection.ipynb`:")
print("   - Carregue os dados de `model2_sequences_labels.pkl`.")
print("   - Treine, avalie e salve o Modelo 2 (TCN).")
print("3. Finalmente, abra o notebook `03_Ensemble_Voting_System.ipynb`:")
print("   - Carregue os modelos treinados 1 e 2.")
print("   - Implemente e avalie o sistema de votação ponderada para a decisão final de fadiga.")

print("\nFim do Notebook de Preparação de Dados.")

# Opcional: Limpeza de variáveis para liberar memória, se necessário, em ambientes com recursos limitados
# del df_features, all_extracted_features, all_sequences, all_labels, X_sequences, y_labels_tcn, X_model1_features, y_labels_model1
# import gc; gc.collect()

--- Resumo da Preparação e Exploração de Dados ---
Este notebook concluiu as seguintes etapas:
1. Configuração do ambiente e importação das bibliotecas necessárias.
2. Carregamento e exploração inicial dos dados de Ground Truth (KSS e PVT-RT).
3. Processamento de todos os vídeos do dataset DROZY, incluindo:
   - Detecção facial utilizando dlib.
   - Extração de 68 landmarks faciais por frame.
   - Cálculo de métricas de fadiga (EAR e MAR) por frame.
   - Total de 497609 frames processados e salvos em 'processed_data/all_video_features.pkl'.
4. Transformação dos dados de nível de frame em sequências temporais e características agregadas por janela:
   - Geradas 11006 sequências de 3 segundos (90 frames) para o Modelo 2 (TCN).
   - Geradas 11006 amostras com características agregadas por janela para o Modelo 1.
   - Dados salvos em 'processed_data/model1_features_labels.pkl' e 'processed_data/model2_sequences_labels.pkl'.

Os dados estão agora prontos e pré-processados para o treinamento